In [2]:
import os
import sys
import time
import scipy
import pandas as pd
import numpy as np
from sklearn import ensemble, linear_model, tree, svm, metrics, model_selection, preprocessing, neighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold

file_path = "alldata.csv"

alldata = pd.read_csv(file_path, sep="\t")

data = alldata.iloc[:,8:].astype(float)

normals = (alldata.condition == 'normal') 
progeria = (alldata.condition == 'hgps')

data_normal = data[normals]
data_prog = data[progeria]

age_normal = alldata[normals].age 
age_prog = alldata[progeria].age

print(data_normal.shape)
print(age_normal.shape)

print(data_prog.shape)
print(age_prog.shape)

(133, 27142)
(133,)
(10, 27142)
(10,)


In [130]:
print('Training started...')
start_time = time.time()

from sklearn.feature_selection import *
from sklearn.decomposition import *
from xgboost import XGBRegressor

n_jobs = 6
n_cv = 5
random_state = 3111696

'''pipe_regressor = Pipeline([
  ('fs', SelectKBest(f_regression)),
  ('regression', linear_model.LinearRegression())
])

parameters = [
    {   
        'fs__k': [5895],
        'regression__normalize': [True]
    }
]'''

'''pipe_regressor = Pipeline([
  ('fs', SelectKBest(f_regression)),
  ('regression', linear_model.ElasticNet())
])

parameters = [
    {   
        'fs__k': [5890],
        'regression__normalize': [True],
        'regression__alpha': [0.0001],
    }
]'''

'''pipe_regressor = Pipeline([
  ('fs', SelectKBest(f_regression)),
  ('regression', svm.SVR())
])

parameters = [
    {   
        'fs__k': [5890, 5895, 5900],
        #'regression__C':[1],
        'regression__kernel':['poly'],
        'regression__degree':[1],
        'regression__epsilon':[6.5, 6.75, 7.0, 7.25, 7.5],
        #'regression__gamma':['auto']
    }
]'''


'''pipe_regressor = Pipeline([
  ('preprocessing', preprocessing.MinMaxScaler()),
  ('fs', SelectKBest(f_regression)),
  ('regression', neighbors.KNeighborsRegressor())
])

parameters = [
    {   
        'fs__k': [1000]
    }
]'''

pipe_regressor = Pipeline([
  ('preprocessing', preprocessing.MinMaxScaler()),
  ('fs', SelectKBest(f_regression)),
  ('regression', svm.LinearSVR())
])

parameters = [
    {   
        'fs__k': [4000, 5000, 6000, 7000],
        #'fs__k': [40, 50, 60, 70, 80, 90, 100],
        #'regression__C': [1.0, 2.0, 3.0],
        #'regression__loss': ['epsilon-insensitive ', 'squared_epsilon_insensitive'],
        #'regression__intercept_scaling': [0.0, 0.1, 0.25, 0.75, 1.0],
        #'regression__max_features': [None, 'auto', 'sqrt', 'log2'],
        #'regression__algorithm': ['auto'],
        #'regression__max_depth': [3, 4, 5],
        #'regression__alpha': [0.00001, 0.0001],
        #'regression__tol': [0.00001, 0.0001, 0.001],
        #'regression__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        #'regression__max_subpopulation': [1],
        #'regression__copy_X': [True],
        #'regression__dual': [True, False]
    }
]

optimized_regressor = GridSearchCV(pipe_regressor, parameters, \
                                       cv=KFold(n_splits=n_cv, shuffle=True, random_state=random_state), \
                                       error_score=0, scoring='r2', verbose=True, n_jobs=n_jobs, \
                                       pre_dispatch="1*n_jobs")

optimized_regressor.fit(data_normal, age_normal)
best_regressor = optimized_regressor.best_estimator_
best_result = optimized_regressor.cv_results_
print(optimized_regressor.best_params_)

best_score = optimized_regressor.best_score_
print("R2 score for training: %.2f" % best_score)

print('Training finished')

end_time = time.time()
print("Time taken: %d" % int(end_time - start_time))

print("Test on Progeria data...")
prediction = best_regressor.predict(data_prog)
test_r2 = metrics.r2_score(age_prog, prediction)
print("R2 score for test: %.2f" % test_r2)

mean_abs_error = np.mean([abs(a - b) for a, b in zip(age_prog, prediction)])
print("Mean absolute error for test: %.2f" % mean_abs_error)

median_abs_error = np.median([abs(a - b) for a, b in zip(age_prog, prediction)])
print("Median absolute error for test: %.2f" % median_abs_error)

## Reproduce paper
## https://genomebiology.biomedcentral.com/articles/10.1186/s13059-018-1599-6#Sec9

Training started...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:    7.0s finished
/home/anupkumar/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/anupkumar/miniconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/anupkumar/miniconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/home/anupkumar/miniconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x

{'fs__k': 6000}
R2 score for training: 0.73
Training finished
Time taken: 7
Test on Progeria data...
R2 score for test: -66.77
Mean absolute error for test: 14.93
Median absolute error for test: 12.18
